## Using BigQuery ML to Predict Taxi Fares

### April 7, 2019, Team 4 
<b>Kush Shah, Pallav Jhaveri, Darshan Shah, Anusha Balaji </b>

In [1]:
!pip install --upgrade google-cloud-bigquery

Requirement already up-to-date: google-cloud-bigquery in /usr/local/envs/py2env/lib/python2.7/site-packages (1.11.2)


In [2]:
from google.cloud import bigquery
client = bigquery.Client()

In [3]:
dataset = bigquery.Dataset(client.dataset('bqml_example'))
dataset.location = 'US'
#client.create_dataset(dataset)

In [4]:
client.get_dataset("bqml_example")

Dataset(DatasetReference(u'bq-tutorial-april-2019', u'bqml_example'))

In [5]:
%load_ext google.cloud.bigquery

In [6]:
%%bigquery
CREATE OR REPLACE MODEL bqml_example.taxi_model OPTIONS(
  model_type='linear_reg', max_iteration=10) AS
SELECT
  miles_range_max as miles_range_max, 
  CAST(average_fare AS FLOAT64) as average_fare,
  CAST(average_fare AS FLOAT64) as label
FROM
  bqml_example.Taxi_Distance_Fares
WHERE
  # remove the miles_range test point to predict/test our model
  miles_range_max != 53.9

""


In [7]:
%%bigquery
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `bqml_example.taxi_model`)

,training_run,iteration,loss,eval_loss,duration_ms,learning_rate
0,0,9,6.808937e-03,None,2963,0.8
1,0,8,5.931148e-02,None,3222,0.8
2,0,7,8.775126e-01,None,3191,1.6
3,0,6,1.013459e+01,None,3909,0.8
4,0,5,9.376790e+01,None,2823,0.8
5,0,4,1.473385e+03,None,4554,1.6
6,0,3,1.516816e+04,None,3338,0.8
7,0,2,1.497204e+05,None,3785,0.8
8,0,1,2.704434e+06,None,3375,0.4
9,0,0,7.545607e+06,None,2064,0.2


In [8]:
%%bigquery
WITH eval_table AS (
SELECT
  miles_range_max as miles_range_max,
  CAST(average_fare AS float64) as average_fare,
  CAST(average_fare AS float64) AS label
FROM
  `bqml_example.Taxi_Distance_Fares` )
SELECT
  *
FROM
  ML.EVALUATE(MODEL `bqml_example.taxi_model`,
    TABLE eval_table)

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.070336,0.007408,5.841280e-10,0.052723,1.0,1.0


In [12]:
%%bigquery
  WITH trip_to_predict AS (
  SELECT
    miles_range_max as miles_range_max,
    CAST(average_fare as float64) as average_fare
  FROM
    `bqml_example.Taxi_Distance_Fares`
  WHERE
    miles_range_max=53.9)
SELECT
  truth.miles_range_max AS miles_range_max,
  CAST(average_fare as float64) as average_fare,
  CAST(predicted_average_fare as float64) as predicted_average_fare
FROM (
  SELECT
    miles_range_max,
    predicted_label AS predicted_average_fare
  FROM
    ML.PREDICT(MODEL `bqml_example.taxi_model`,
      table trip_to_predict) ) AS predict
JOIN (
  SELECT
    miles_range_max,
    CAST(average_fare as float64) as average_fare
  FROM
    trip_to_predict) AS truth
ON
  predict.miles_range_max = truth.miles_range_max

,miles_range_max,average_fare,predicted_average_fare
0,53.9,5386.99,5387.362983
